In [13]:
pip install --upgrade scikit-learn

Requirement already up-to-date: scikit-learn in c:\users\utilisateur\anaconda3\lib\site-packages (0.24.1)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import joblib
import pandas as pd 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import roc_auc_score, f1_score,classification_report
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from joblib import dump

In [2]:
#Import de la pipeline utilisée dans l'API et du jeu de données contenant les commentaites. Affichage de la
#pipeline pour étudier son contenu

modele=joblib.load("sentiment_pipe.joblib")
df=pd.read_csv("comments_train.csv")
modele

C:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\utilisateur\anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator Pipeline from version 0.23.2 when using version 0.24.1. This might lead to breaking code or invalid resu

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, gamma=0.001))])

In [3]:
#affichage du dataset, on voit un net déséquilibre entre les réponse
#positive et négéative
df = pd.read_csv('comments_train.csv')
X,y = df['comment'], df['sentiment'] 
df["sentiment"].value_counts()

Positive    1019
Negative     598
Name: sentiment, dtype: int64

In [4]:
#on définit ypred et on labellise de manière binaire
#la colonne sentiment du dataset, pour pouvoir faire l'auc
y_pred = modele.predict(X)
y=df["sentiment"].replace({"Positive":1, "Negative":0}).values
resume=classification_report(y_pred, y)
roc_score=roc_auc_score(y_pred, y)
print("AUC : {}".format(roc_score))
print()
print(resume)

AUC : 0.7220142706749327

              precision    recall  f1-score   support

           0       0.26      0.76      0.39       206
           1       0.95      0.69      0.80      1411

    accuracy                           0.70      1617
   macro avg       0.61      0.72      0.59      1617
weighted avg       0.86      0.70      0.75      1617



In [ ]:
#on peut voir ci-dessu le score optenu avec 
#les réponse négative étant mal reconnus contrairement au positive

In [44]:
#on réentraine le modèle a partir du dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
modele.fit(X_train,y_train)
y_pred=modele.predict(X_test)
print(classification_report(y_test, y_pred,zero_division=1))
y_pred2=modele.decision_function(X_test)
roc_score=roc_auc_score(y_test,y_pred2)
print("AUC : ",roc_score)

              precision    recall  f1-score   support

           0       0.85      0.74      0.79       202
           1       0.85      0.92      0.89       332

    accuracy                           0.85       534
   macro avg       0.85      0.83      0.84       534
weighted avg       0.85      0.85      0.85       534

AUC :  0.9279046880591674


In [46]:
#on peut voir que le réentrainement a amélioré la prediction, 
#notament sur la prédiction négative marqué par 0 ainsi que l'auc

In [47]:
# Afin de permettre l'amélioration de la pipeline, on affiche ses paramètres. 

modele.get_params()

{'memory': None,
 'steps': [('tfidf', TfidfVectorizer()), ('clf', SVC(C=1000, gamma=0.001))],
 'verbose': False,
 'tfidf': TfidfVectorizer(),
 'clf': SVC(C=1000, gamma=0.001),
 'tfidf__analyzer': 'word',
 'tfidf__binary': False,
 'tfidf__decode_error': 'strict',
 'tfidf__dtype': numpy.float64,
 'tfidf__encoding': 'utf-8',
 'tfidf__input': 'content',
 'tfidf__lowercase': True,
 'tfidf__max_df': 1.0,
 'tfidf__max_features': None,
 'tfidf__min_df': 1,
 'tfidf__ngram_range': (1, 1),
 'tfidf__norm': 'l2',
 'tfidf__preprocessor': None,
 'tfidf__smooth_idf': True,
 'tfidf__stop_words': None,
 'tfidf__strip_accents': None,
 'tfidf__sublinear_tf': False,
 'tfidf__token_pattern': '(?u)\\b\\w\\w+\\b',
 'tfidf__tokenizer': None,
 'tfidf__use_idf': True,
 'tfidf__vocabulary': None,
 'clf__C': 1000,
 'clf__break_ties': False,
 'clf__cache_size': 200,
 'clf__class_weight': None,
 'clf__coef0': 0.0,
 'clf__decision_function_shape': 'ovr',
 'clf__degree': 3,
 'clf__gamma': 0.001,
 'clf__kernel': 'rbf',

In [50]:
#pour améliorer le modèle on fait une gridSearch sur ses paramètre, 
#afin de trouver quel sont les meilleur paramètre pour optimiser le modèle
tuned_parameters = { 
    "clf__kernel": ["sigmoid","rbf","poly"],
    "clf__C" : [1000,10e5,10],
    "clf__gamma" : [0.001,10e-3,0.1],
    "clf__random_state":[42]}
clf = GridSearchCV(
        modele, tuned_parameters, n_jobs=1, verbose=3)
clf.fit(X_train, y_train)

parametre_optimale = clf.best_params_
print(parametre_optimale)

print("les résultats obtenus pour les métriques sont : ")
y_pred=modele.predict(X_test)
print(classification_report(y_test, y_pred,zero_division=1))
y_pred2=clf.decision_function(X_test)
roc_score=roc_auc_score(y_test,y_pred2)
print("AUC : ",roc_score)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.4s
[CV 2/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.3s
[CV 3/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.3s
[CV 4/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.6s
[CV 5/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=sigmoid, clf__random_state=42; total time=   0.2s
[CV 1/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.3s
[CV 2/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.3s
[CV 3/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.3s
[CV 4/5] END clf__C=1000, clf__gamma=0.001, clf__kernel=rbf, clf__random_state=42; total time=   0.3s


[CV 4/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=sigmoid, clf__random_state=42; total time=   0.3s
[CV 5/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=sigmoid, clf__random_state=42; total time=   0.4s
[CV 1/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=rbf, clf__random_state=42; total time=   0.4s
[CV 2/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=rbf, clf__random_state=42; total time=   0.4s
[CV 3/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=rbf, clf__random_state=42; total time=   0.4s
[CV 4/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=rbf, clf__random_state=42; total time=   0.3s
[CV 5/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=rbf, clf__random_state=42; total time=   0.4s
[CV 1/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=poly, clf__random_state=42; total time=   0.4s
[CV 2/5] END clf__C=1000000.0, clf__gamma=0.1, clf__kernel=poly, clf__random_state=42; total time=   0.4s
[CV 3/5] END clf__C=1000000.0, clf__gamma=0.1

In [51]:
print(modele)
print(parametre_optimale)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, gamma=0.001))])
{'clf__C': 1000, 'clf__gamma': 0.001, 'clf__kernel': 'sigmoid', 'clf__random_state': 42}


In [56]:
#on voit ci-dessu les meilleur paramètre
#on peut donc créer une nouvelle pipeline en se basant dessus
#pour améliorer le modèle

modele_opti = Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', SVC(C=1000, kernel="sigmoid", gamma=0.001, random_state = 42))])

modele_opti.fit(X_train, y_train)
y_pred = modele_opti.predict(X_test)
roc_score_1 = roc_auc_score(y_test, y_pred)
print(classification_report(y_test, y_pred,zero_division=1))
roc_score_2 = roc_auc_score(y_test, y_pred2)

print ("AUC : ", roc_score_1)
print("AUC avec y_pred2 : ", roc_score_2)





              precision    recall  f1-score   support

           0       0.84      0.75      0.80       202
           1       0.86      0.92      0.89       332

    accuracy                           0.85       534
   macro avg       0.85      0.83      0.84       534
weighted avg       0.85      0.85      0.85       534

AUC :  0.834068949063581
AUC avec y_pred2 :  0.930320291065251


In [57]:
#on peut voir que l'auc est amélioré, et qu'avec
#la fonction de decision, on optien un score de 0,93

In [58]:
#sauvegarde de la nouvelle pipeline dans un nouveau fichier joblib
dump(modele_opti, "pipeline_optimiser.joblib")

['pipeline_optimiser.joblib']